# CNTL temperature dynamic system

## Import package

In [17]:
import numpy as np
import joblib as jl

from matplotlib import pyplot as plt

## Load data

In [18]:
case  = 'CNTL'
fname = f'/home/b11209013/2024_Research/MPAS/LRF/Tseries_file/{case}.joblib'

data = jl.load(fname)
tpc1 = data['pc1']['t']
tpc2 = data['pc2']['t'] # unit in K

# time 
time = np.linspace(0, 32, 33)*0.25 # unit in day

## Compute finite different of the PCs

### Compute the derivartive of the PC time series

In [19]:
def forward_diff(data):
    data_for_diff = np.roll(data, -1)
    derv = (data_for_diff - data)/0.25

    return derv

tpc1_dt = forward_diff(tpc1) # unit in K/day
tpc2_dt = forward_diff(tpc2)

## Constructing dynamic system

In [20]:
t_tend = np.stack([tpc1_dt, tpc2_dt], axis=0)
t_vec  = np.stack([tpc1, tpc2], axis=0)

M  = (t_tend @ t_vec.T) @ np.linalg.inv(t_vec @ t_vec.T) # unit in 1/day

print(M)

[[-0.67848758 -1.38715188]
 [ 0.74551636  0.44750178]]


## Computing e-folding time scale

In [21]:
eigvals, eigvec = np.linalg.eig(M)
print('Eigenvalue of the LIM: \n', eigvals)
print('Eigenvectors of the LIM: \n', eigvec)

# growth/decay time
gd_time = 1/np.real(eigvals)
period  = 2*np.pi / np.imag(eigvals)
print('\n')
print('Growth/Decay time of CCKW: ', gd_time[0])
print('Period of CCKW: ', period[0])

Eigenvalue of the LIM: 
 [-0.1154929+0.84686564j -0.1154929-0.84686564j]
Eigenvectors of the LIM: 
 [[ 0.80649254+0.j          0.80649254-0.j        ]
 [-0.3273261 -0.49236917j -0.3273261 +0.49236917j]]


Growth/Decay time of CCKW:  -8.658541108689432
Period of CCKW:  7.419341416794059
